In [20]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from collections import Counter, defaultdict
import itertools

import load_results

pd.set_option('display.max_colwidth', 180)
pd.set_option('expand_frame_repr', False)

In [21]:
relevant_mn_col = 'spellchecked' # 'canon'

In [22]:
csvfile = '../proc_data_phase0/verification/all_responses_round0-3_verified.csv'
df = load_results.load_cleaned_results(csvfile)

total_objects=len(df)
print("Total objects: ", total_objects)
df.columns

Total objects:  25315


Index(['vg_img_id', 'cat', 'synset', 'vg_obj_name', 'responses_r0', 'opt-outs',
       'vg_domain', 'top_response_domain_r0', 'responses_domains_r0', 'url',
       'sample_type', 'responses_domains_r1', 'responses_r1',
       'top_response_domain_r1', 'responses_domains_r2', 'responses_r2',
       'top_response_domain_r2', 'responses_domains_r3', 'responses_r3',
       'top_response_domain_r3', 'vg_object_id', 'all_responses', 'clean',
       'canon', 'spellchecked', 'spellchecked_min2', 'verified'],
      dtype='object')

#### Load MN --> VG_aliases

In [23]:
mn2alias = dict([line.strip().split("\t") for line in open("objects_vocab-442_aliasmap.txt")])

In [28]:
df['mn_name'] = df[relevant_mn_col].apply(lambda x: mn2alias.get(x.most_common(1)[0][0], 
                                                         x.most_common(1)[0][0]))
vocab = set(df['mn_name'].values)

### Load system predictions

In [29]:
csvfile = '../papers/ACL2020/material_experiments/FRCNN--VG1600--VGMN_MN.tsv'
csvfile2 = '../papers/ACL2020/material_experiments/FRCNN--VG1600_VG.tsv'

dfsys = pd.read_csv(csvfile, 
    sep="\t")

total_objects=len(dfsys)
print("Total objects: ", total_objects)
dfsys.columns

dfsys2 = pd.read_csv(csvfile2, 
    sep="\t")

total_objects=len(dfsys2)
print("Total objects: ", total_objects)
dfsys2.columns

Total objects:  1071
Total objects:  1071


Index(['image_id', 'category', 'mn_name', 'adequacy', 'pred_name',
       'adequacy.1', 'pred_score_converted'],
      dtype='object')

In [30]:
df.query('vg_img_id == 32')

,vg_img_id,cat,synset,vg_obj_name,responses_r0,opt-outs,vg_domain,top_response_domain_r0,responses_domains_r0,url,...,responses_r3,top_response_domain_r3,vg_object_id,all_responses,clean,canon,spellchecked,spellchecked_min2,verified,mn_name
21087,32,"instrumentality, instrumentation",furnishing.n.02,cabinet,"{'cabinet': 2, 'wood': 1, 'sink': 1, 'cabinets': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}",{'#bbox': 1},home,home,"{'home': 0.25, '$wood': 0.125, '$sink': 0.125, '$cabinets': 0.125, '$cabinetry': 0.125, '$modern kitchen': 0.125, '$wall': 0.125}",http://object-naming-amore.upf.edu//32_1058643_seed_ambiguous.png,...,"{'cabinet': 4, 'cabinets': 2, 'cupboards': 2, 'cupboard': 1}",home,1058643,"Counter({'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1})","{'cabinet': 15, 'cabinets': 9, 'cupboard': 3, 'wood': 2, 'cupboards': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","{'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","{'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2, 'sink': 1, 'cabinetry': 1, 'modern kitchen': 1, 'wall': 1}","{'cabinet': 15, 'cabinets': 9, 'cupboard': 5, 'wood': 2}","{'cabinets': {'cluster': ('cabinet', 'cabinets', 'cupboard'), 'can_be_same_object': ('cabinet', 'cabinets', 'cupboard'), 'adequacy': 1.0, 'inadequacy_type': None, 'cluster_id':...",cabinet


In [31]:
def get_error(topname,predname,verified,response):
    if predname == topname:
        return 'hit'
    if predname not in verified:
        if predname in response:
            return 'low_count'
        else:
            return 'off???'
    if topname in verified[predname]['can_be_same_object']:
        return 'adequate'
    # CS: This would consider names with adequacy < 1.0 as inadequate, but this is wrong, isn't it? 
    # But our threshold was adequacy > 0.5 ==> adequate
    # e.g.: (2417522, vegetable): 'adequacy': 0.6428571428571428, 'inadequacy_type': 'bounding box'
    if not verified[predname]['inadequacy_type']:
        return 'adequate_diff_object'
    
    return verified[predname]['inadequacy_type']

In [50]:
def get_error_carina(topname, predname, verified, response, adequacy_threshold=0.5):
    if predname == topname:
        return 'hit'
    if predname not in verified:
        if predname in response:
            return 'low_count'
        else:
            return 'off???'
    if topname in verified[predname]['can_be_same_object'] and verified[predname]['adequacy'] > adequacy_threshold:
        return 'adequate'
    # CS: Consider names of other objects with adequacy > 0.5 as adequate, 
    # e.g.: (2417522, vegetable): 'adequacy': 0.6428571428571428, 'inadequacy_type': 'bounding box'
    if verified[predname]['adequacy'] > adequacy_threshold:
        return 'adequate_diff_object'
    
    return verified[predname]['inadequacy_type']

#### Test on top-MN ground truth

In [33]:
def test_on_mn(df, dfsys, dfsys2, error_func, col_key):
    """
    @param col_key: 'mn_name' or 'vg_obj_name'
    """
    hum_error_cats = Counter()
    sys_error_cats = Counter()
    sys_error_cats2 = Counter()

    for ix,row in dfsys.iterrows():
        rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
        resp = rowh[relevant_mn_col]
        topname = rowh[col_key]
        verif = rowh['verified']

        #print(topname)
        for name in resp:
            if name in vocab:
                err = error_func(topname,name,verif,resp)
                hum_error_cats[err] += resp[name]

        sys_err = error_func(topname,row['pred_name'],verif,resp)
        sys_error_cats[sys_err] += 1


    for ix,row in dfsys2.iterrows():
        rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
        resp = rowh[relevant_mn_col]
        topname = rowh[col_key]
        verif = rowh['verified']
        sys_err = error_func(topname,row['pred_name'],verif,resp)
        sys_error_cats2[sys_err] += 1
        
    return hum_error_cats, sys_error_cats, sys_error_cats2

### Systems against human upper bound, version I

In [53]:
error_func = get_error
col_key = "mn_name"
hum_error_cats, sys_error_cats, sys_error_cats2 = test_on_mn(df, dfsys, dfsys2, error_func, col_key)
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
    ["humans"]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["FRCNN, no finetuning"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["FRCNN, finetuning on MN"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]

In [54]:
sys_error_cats

Counter({'hit': 862,
         'off???': 77,
         'adequate': 94,
         'low_count': 19,
         'adequate_diff_object': 6,
         'bounding box': 12,
         'visual': 1})

In [46]:
pd.DataFrame(table,columns=['(442-vocab)']+relevant_cats)

,(442-vocab),hit,adequate,low_count,bounding box,off???
0,humans,0.763750,0.144260,0.038469,0.042247,0.000000
1,"FRCNN, no finetuning",0.747899,0.130719,0.024276,0.018674,0.070028
2,"FRCNN, finetuning on MN",0.804855,0.087768,0.017740,0.011204,0.071895


### Systems against human upper bound, version II

In [51]:
error_func = get_error_carina
col_key = "mn_name"
hum_error_cats, sys_error_cats, sys_error_cats2 = test_on_mn(df, dfsys, dfsys2, error_func, col_key)
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
    ["humans"]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["FRCNN, no finetuning"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["FRCNN, finetuning on MN"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]

In [52]:
pd.DataFrame(table,columns=['(442-vocab)']+relevant_cats)

,(442-vocab),hit,adequate,low_count,bounding box,off???
0,humans,0.763750,0.139234,0.038469,0.040329,0.000000
1,"FRCNN, no finetuning",0.747899,0.127918,0.024276,0.016807,0.070028
2,"FRCNN, finetuning on MN",0.804855,0.084967,0.017740,0.009337,0.071895


In [49]:
sys_error_cats

Counter({'hit': 862,
         'off???': 77,
         'adequate': 94,
         'low_count': 19,
         'adequate_diff_object': 8,
         'bounding box': 10,
         'visual': 1})

#### Test on VG ground truth

In [23]:
hum2_error_cats = Counter()
sys3_error_cats = Counter()
sys4_error_cats = Counter()


for ix,row in dfsys.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh[relevant_mn_col]
    topname = rowh['vg_obj_name']
    verif = rowh['verified']

    #print(topname)
    for name in resp:
        if name in vocab:
            err = get_error(topname,name,verif,resp)
            hum2_error_cats[err] += resp[name]
        
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys3_error_cats[sys_err] += 1
    
for ix,row in dfsys2.iterrows():
    
    rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
    resp = rowh[relevant_mn_col]
   
    topname = rowh['vg_obj_name']
    verif = rowh['verified']
    sys_err = get_error(topname,row['pred_name'],verif,resp)
    sys4_error_cats[sys_err] += 1

In [24]:
relevant_cats = ['hit','adequate','low_count','bounding box','off???']
table = [
   # ["VG","human",""]+[hum2_error_cats[t]/sum(hum2_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "-"]+[sys4_error_cats[t]/sum(sys4_error_cats.values()) for t in relevant_cats],
   # ["VG","FRCNN", "+"]+[sys3_error_cats[t]/sum(sys3_error_cats.values()) for t in relevant_cats],
    ["MN","human",""]+[hum_error_cats[t]/sum(hum_error_cats.values()) for t in relevant_cats],
    ["MN","FRCNN", "-"]+[sys_error_cats2[t]/sum(sys_error_cats2.values()) for t in relevant_cats],
    ["MN","FRCNN", "+"]+[sys_error_cats[t]/sum(sys_error_cats.values()) for t in relevant_cats]
]
rdf = pd.DataFrame(table,columns=['Test on','model', 'finetuning']+relevant_cats)

In [25]:
rdf.round(3)

,Test on,model,finetuning,hit,adequate,low_count,bounding box,off???
0,MN,human,,0.759,0.146,0.042,0.044,0.000
1,MN,FRCNN,-,0.750,0.122,0.023,0.019,0.077
2,MN,FRCNN,+,0.789,0.096,0.018,0.011,0.079


### Get "Off"s

In [68]:
def off_candidates(df, dfsys, dfsys2, error_func, col_key):
    """
    @param col_key: 'mn_name' or 'vg_obj_name'
    """
    hum_error_cats = defaultdict(list)
    sys_error_cats = defaultdict(list)
    sys_error_cats2 =defaultdict(list)

    for ix,row in dfsys.iterrows():
        rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
        resp = rowh[relevant_mn_col]
        topname = rowh[col_key]
        verif = rowh['verified']
        #print(topname)
        for name in resp:
            if name in vocab:
                err = error_func(topname,name,verif,resp)
                hum_error_cats[err].append((topname,name))

        sys_err = error_func(topname,row['pred_name'],verif,resp)
        sys_error_cats[sys_err].append((topname,row['pred_name']))


    for ix,row in dfsys2.iterrows():
        rowh = df.query('vg_img_id == %d' % row['image_id']).iloc[0]
        resp = rowh[relevant_mn_col]
        topname = rowh[col_key]
        verif = rowh['verified']
        sys_err = error_func(topname,row['pred_name'],verif,resp)
        sys_error_cats2[sys_err].append((topname,row['pred_name']))
        
    return hum_error_cats, sys_error_cats, sys_error_cats2

In [69]:
error_func = get_error_carina
hum_error_cats, sys_error_cats, sys_error_cats2 = off_candidates(df, dfsys, dfsys2, error_func, "mn_name")

In [81]:
err_cat = "off???"
for (top_pred,cnt) in Counter(sys_error_cats2[err_cat]).items():
    print(top_pred, cnt)

('dress', 'woman') 1
('bridge', 'sidewalk,side walk') 1
('cheetah', 'giraffe') 1
('table', 'ground') 1
('hotdog', 'hot dog,hotdog') 5
('cow', 'animal') 1
('dress', 'man') 1
('truck', 'van') 1
('pan', 'bowl') 1
('child', 'boy') 2
('planter', 'bench') 1
('jacket', 'man') 3
('sandwich', 'hot dog,hotdog') 1
('cake', 'fruit') 1
('woman', 'girl') 1
('bench', 'fence') 1
('boy', 'floor') 1
('boat', 'kayak') 1
('jacket', 'teddy bear') 1
('cat', 'window') 1
('van', 'car') 1
('man', 'woman') 2
('house', 'wall') 1
('girl', 'woman') 2
('jacket', 'person') 1
('bun', 'hot dog,hotdog') 1
('horse', 'man') 1
('surfer', 'woman') 1
('counter', 'floor') 2
('pizza', 'plate') 2
('waffle', 'plate') 1
('truck', 'ramp') 1
('truck', 'window') 1
('sink', 'floor') 1
('curtain', 'window') 1
('horse', 'ground') 1
('car', 'fence') 1
('skater', 'boy') 1
('boy', 'baby') 1
('dog', 'stand') 1
('cake', 'box') 1
('chair', 'couch') 1
('donut', 'donut,doughnut') 2
('mustard', 'sauce') 1
('shirt', 'jacket') 3
('toilet', 'seat

### Playground

In [14]:
error_cats = Counter()

for ix,row in df.iterrows():
    resp = row[relevant_mn_col]
    topname = row['mn_name']
    #print(topname)
    for name in resp:
        if name == topname:
            error_cats['hit'] += resp[name]
        elif name not in row['verified']:
            error_cats['low_count'] += resp[name]
        else:
            if topname in row['verified'][name]['can_be_same_object']:
                error_cats['adequate_same_object'] += resp[name]
            else:
                # CS: This would consider names with adequacy < 1.0 as inadequate, but this is wrong, isn't it? 
                # But our threshold was adequacy > 0.5 ==> adequate
                # e.g.: (2417522, vegetable): 'adequacy': 0.6428571428571428, 'inadequacy_type': 'bounding box'
                if not row['verified'][name]['inadequacy_type']:
                    error_cats['adequate_diff_object'] += resp[name]
                else:
                     error_cats[row['verified'][name]['inadequacy_type']] += resp[name]                 
                    
                    #print('img:',row['vg_img_id'],'--entry-level:',topname)
                    #print('less-preferred:',name)
                    #print(row['verified'][name])
            
    #if ix > 5:       
    #    break
        

man
jacket
giraffe
giraffe
sandwich
van
car
desk
zebra
child
woman
jacket
cat
truck
pizza
bed
bridge
horse
truck
curtain
man
shirt
salad
cat
dugout
motorcycle
bench
bird
bathtub
train
shirt
boy
pizza
woman
airplane
truck
giraffe
curtain
shirt
dog
plane
cake
house
girl
man
train
airplane
robe
horse
giraffe
pizza
skateboarder
couch
airplane
cat
bed
man
house
airplane
plane
lamp
cow
giraffe
dog
woman
pizza
shirt
floor
man
couch
house
donut
man
dog
zebra
duck
jacket
table
cake
woman
jacket
boy
cat
horse
floor
motorcycle
pizza
girl
bed
toilet
zebra
giraffe
airplane
pizza
table
house
woman
car
shirt
man
horse
pizza
bird
bench
hotdog
boy
child
train
bench
shirt
motorcycle
pier
child
horse
zebra
lettuce
jacket
bed
bench
airplane
dog
girl
sandwich
truck
zebra
duck
dress
man
man
motorcycle
salad
man
hood
horse
bathtub
seat
surfer
train
shirt
sheep
woman
knife
horse
bench
airplane
table
skater
cat
dog
sink
couch
shirt
train
woman
desk
cake
shirt
counter
pizza
bed
cat
train
girl
cake
giraffe
count

woman
sandwich
horse
man
shirt
motorcycle
chair
train
horse
airplane
girl
tie
skier
bench
food
bottle
cow
cat
woman
cabinet
castle
skier
bed
cucumber
truck
suit
train
train
motorcycle
food
skier
man
bridge
jacket
man
dugout
man
shirt
desk
woman
motorcycle
cat
pizza
pizza
cow
child
cat
china
horse
car
grill
giraffe
table
train
woman
bridge
cat
jacket
bed
woman
car
shirt
umpire
bed
horse
zebra
food
airplane
man
woman
bed
umbrella
table
pizza
horse
man
bowl
dress
bathtub
pizza
giraffe
woman
man
couch
boy
airplane
cat
dress
couch
skateboarder
chair
cupcake
truck
cat
tray
man
shirt
cow
cat
zebra
man
counter
woman
soup
shirt
bed
giraffe
spoon
boat
woman
plane
boy
shirt
couch
skater
train
pizza
giraffe
cat
desk
boy
girl
desk
train
woman
pizza
giraffe
spoon
garage
boat
plane
cat
cat
coat
swan
train
table
man
horse
bridge
cat
pizza
apple
curtain
plane
cat
carpet
coat
dugout
skier
table
cow
woman
train
hotdog
woman
giraffe
rice
boy
hat
shirt
man
train
woman
zebra
house
dog
cake
couch
bed
chair
c

jacket
train
bread
hat
boy
cat
garage
cat
counter
girl
truck
tent
bread
bed
girl
shirt
cat
table
truck
horse
bed
blender
car
pizza
train
shirt
pizza
girl
car
giraffe
motorcycle
vase
bench
woman
horse
jacket
motorcycle
suit
truck
pizza
cat
dog
woman
curtain
chair
woman
sheep
car
man
man
bathtub
jacket
dugout
snowboarder
zebra
boy
shirt
pizza
plane
sink
table
shirt
cupcake
horse
giraffe
woman
sheep
couch
house
sheep
bed
cat
plane
giraffe
woman
shirt
pizza
man
building
curtain
cow
bird
giraffe
soup
car
dress
hat
boat
couch
cow
bench
train
hotdog
man
man
woman
salad
man
bench
truck
shirt
parrot
horse
floor
horse
dog
woman
cake
shirt
bowl
zebra
bench
table
dress
cake
boy
cat
woman
plane
vase
woman
bird
boy
shirt
curtain
pizza
girl
giraffe
child
desk
cat
jacket
woman
toilet
sandwich
zebra
train
dog
desk
cat
dress
baby
cake
jacket
zebra
desk
bird
boat
giraffe
bathtub
horse
bench
train
girl
man
boy
jacket
pizza
horse
jacket
man
airplane
bowl
bird
chair
couch
pizza
giraffe
girl
cat
bed
shirt
be

skateboarder
carpet
sandwich
dog
bowl
table
baseball player
dog
burger
zebra
woman
boat
jacket
motorcycle
jacket
bowl
airplane
food
man
carpet
dog
zebra
child
table
airplane
motorcycle
giraffe
dog
shirt
pizza
pizza
cat
desk
chair
horse
woman
train
man
bed
vase
hotdog
jacket
woman
motorcycle
horse
chair
man
zebra
sink
man
bed
airplane
dress
woman
zebra
soup
giraffe
building
desk
zebra
broccoli
cat
jacket
shirt
desk
baseball player
dress
couch
pizza
boy
bed
bridge
cat
plane
man
giraffe
boy
table
runway
pancake
shirt
zebra
woman
giraffe
house
man
train
bird
man
carpet
woman
spatula
jacket
bench
van
hamster
man
couch
house
bun
bed
jacket
bird
bench
cat
train
food
man
hotdog
couch
bench
woman
cow
shirt
sandwich
bench
woman
bed
boat
giraffe
horse
cake
man
motorcycle
car
vase
shirt
zebra
man
dog
horse
motorcycle
desk
street
butterfly
shirt
child
pizza
sandwich
boat
horse
zebra
woman
cat
dugout
chair
woman
shirt
horse
window
truck
train
hotdog
woman
sheep
cat
tray
dress
shelf
carpet
boy
woman


train
jacket
pizza
airplane
cake
suit
bird
woman
airplane
tent
table
giraffe
child
suit
car
jet
woman
man
man
giraffe
sandwich
woman
motorcycle
skateboarder
sandwich
truck
dog
tie
sheep
bench
juice
woman
donut
woman
table
car
desk
shirt
man
person
cow
rug
pizza
bowl
train
zebra
bed
shirt
dog
house
curtain
tie
bed
mouse
woman
truck
sandwich
bed
plane
rug
giraffe
blender
jacket
snowboarder
woman
airplane
car
cake
desk
woman
desk
shirt
dog
shirt
bridge
boy
car
pizza
house
train
bench
bench
horse
dress
woman
cat
car
jacket
bus stop
pizza
curtain
cow
carpet
truck
skater
zebra
pizza
child
child
man
dog
woman
shirt
pizza
court
zebra
airplane
giraffe
motorcycle
jacket
bed
pizza
horse
scooter
skateboarder
woman
counter
tie
giraffe
man
pretzel
tent
train
woman
motorcycle
catcher
cat
chair
house
cow
man
boat
turkey
sandwich
pizza
man
desk
jet
bowl
cat
plane
shirt
boy
table
cake
giraffe
horse
catcher
shirt
airplane
table
couch
dugout
curtain
dress
bench
train
sheep
boy
man
pizza
giraffe
girl
train

...

zebra
cabinet
woman
shirt
cake
flower
desk
skater
train
airplane
cake
giraffe
house
woman
bridge
boat
dog
shirt
zebra
jersey
counter
bench
horse
man
pizza
motorcycle
dog
man
blender
jacket
counter
woman
donut
woman
sheep
motorcycle
scooter
bread
shirt
dog
curtain
woman
bathtub
skier
giraffe
horse
sandwich
shirt
woman
couch
giraffe
house
man
train
airplane
bench
shirt
woman
cat
woman
rhubarb
cow
motorcycle
pizza
car
dog
couch
man
giraffe
woman
woman
man
airplane
jacket
salad
boy
woman
hotdog
shirt
couch
bowl
boy
scooter
man
airplane
bench
pizza
airplane
plane
plane
shirt
woman
desk
bird
pizza
bird
bathtub
woman
horse
man
cake
shirt
man
train
woman
woman
grill
man
dog
cake
train
table
jacket
house
jacket
train
woman
giraffe
airplane
desk
pizza
woman
shirt
boy
plane
girl
giraffe
woman
shirt
airplane
zebra
counter
train
horse
pasta
desk
shirt
pizza
zebra
dog
couch
train
man
boy
desk
woman
man
sandwich
cat
desk
jacket
truck
truck
couch
hotdog
woman
horse
zebra
shed
shirt
train
woman
train
m

shirt
chair
man
skater
man
bed
man
kettle
train
cat
truck
carpet
boy
bench
boy
temple
giraffe
car
jacket
car
carpet
car
giraffe
man
laptop
cat
bread
jacket
bus
food
bed
woman
carpet
boy
cat
bench
shirt
man
boy
man
train
train
shirt
bench
airplane
skier
giraffe
man
horse
shirt
train
zebra
girl
van
highchair
man
motorcycle
dress
floor
desk
motorcycle
rice
truck
bread
man
cabinet
shirt
motorcycle
giraffe
soccer player
sandwich
car
man
airplane
desk
cow
broccoli
bridge
skier
dress
trailer
boat
horse
couch
table
dog
shirt
airplane
bed
giraffe
man
man
dog
jet
house
girl
robe
sandwich
cow
bear
desk
salad
motorcycle
zebra
woman
shirt
bowl
baseball player
desk
jacket
pizza
horse
train
giraffe
counter
woman
bird
sheep
man
bird
shirt
train
horse
boat
shirt
man
man
car
zebra
food
shirt
boy
sink
woman
bed
shelf
train
tennis player
restaurant
man
pizza
giraffe
shirt
jacket
pizza
man
desk
fork
car
counter
bird
horse
bread
couch
shirt
dresser
truck
giraffe
zebra
boy
bowl
jacket
giraffe
pizza
airplane


couch
truck
dog
sandwich
cat
horse
motorcycle
bathtub
sandwich
rug
dog
desk
snowboarder
bed
man
bench
shirt
car
dog
building
dog
woman
lamp
giraffe
woman
bench
pizza
jacket
bench
child
car
giraffe
pizza
couch
jacket
carpet
woman
airplane
zebra
desk
vase
bed
dog
man
pizza
spoon
car
airplane
man
horse
jacket
sail
sandwich
motorcycle
woman
dog
cow
tie
boy
clock
truck
cow
picture
cow
pizza
train
man
dress
bench
car
cat
carpet
bench
bowl
plane
boy
giraffe
counter
table
donut
jacket
house
giraffe
desk
jacket
woman
house
motorcycle
truck
bird
surfer
curtain
motorcycle
jacket
table
pizza
cat
house
man
counter
train
giraffe
table
plane
man
bench
train
jacket
dog
hotdog
sidewalk
zebra
man
bench
bird
jacket
train
desk
man
tent
dog
cake
woman
woman
cow
chair
cat
car
cow
shirt
cat
woman
dress
zebra
man
table
motorcycle
food
house
plane
zebra
shirt
motorcycle
man
building
woman
hotdog
dog
rug
woman
bench
cat
dress
bottle
scooter
meat
dog
train
bridge
bed
sheep
man
plane
tie
cake
bookshelf
desk
toast

bed
house
dog
bench
rug
airplane
shirt
cat
horse
tub
skier
train
cat
airplane
plane
person
giraffe
truck
jacket
motorcycle
man
vase
floor
woman
bed
shore
food
house
zebra
dress
zebra
woman
desk
van
van
giraffe
curtain
giraffe
man
shirt
giraffe
cat
man
airplane
shirt
goal
bed
rug
desk
zebra
airplane
bull
bird
building
cake
shirt
shirt
table
dog
skier
train
pizza
zebra
giraffe
cat
salad
man
taxi
cabinet
child
jacket
sink
bed
soccer player
plane
bird
horse
umbrella
car
desk
tie
train
giraffe
skateboarder
hotdog
fork
boat
desk
pizza
mouse
shirt
sheep
woman
zebra
jacket
jet
cow
girl
bench
boy
child
shirt
food
dog
giraffe
desk
truck
airplane
bridge
woman
car
bird
table
desk
shirt
horse
door
dugout
boy
woman
horse
boat
bird
woman
pitcher
cabinet
glove
boy
truck
pizza
zebra
train
desk
bench
dress
dog
boat
train
couch
giraffe
man
shirt
zebra
table
shirt
hotdog
cat
counter
truck
bed
bench
skateboarder
cat
curtain
boat
pizza
dress
girl
desk
woman
bridge
man
giraffe
jacket
motorcycle
pizza
shirt
p

bench
bench
cat
giraffe
chair
table
girl
cow
man
train
boy
train
dugout
cat
couch
giraffe
knife
motorcycle
boat
skateboarder
giraffe
table
woman
keyboard
airplane
sheep
bowl
zebra
woman
zebra
airplane
girl
child
couch
tent
girl
motorcycle
woman
man
plane
giraffe
desk
train
train
woman
woman
zebra
cat
horse
man
car
boat
cat
bed
chair
man
car
cow
car
table
woman
bed
cat
bird
man
child
cabinet
motorcycle
woman
horse
cat
table
boat
horse
knife
car
man
couch
horse
woman
knife
motorcycle
dog
train
woman
man
dog
man
train
dog
knife
desk
bridge
truck
giraffe
couch
bench
plane
woman
man
motorcycle
desk
man
table
woman
motorcycle
cat
boy
ottoman
bike
girl
dog
cat
motorcycle
truck
skateboarder
rug
shower
desk
truck
train
dog
horse
boat
desk
man
giraffe
man
desk
cat
dugout
cow
cow
car
bowl
man
train
dog
carpet
train
cow
desk
man
plane
man
dog
man
mouse
curtain
truck
girl
couch
bed
train
giraffe
woman
boat
boy
giraffe
man
giraffe
boat
dog
beach
table
girl
airplane
horse
rug
table
train
skateboarder

In [53]:
for t in error_cats:
    print(t,error_cats[t]/sum(error_cats.values()))

hit 0.6897080347511604
adequate_same_object 0.160786779877986
bounding box 0.04808385216881437
low_count 0.09046536297595975
other 0.0007788127134461568
adequate_diff_object 0.008894802096528017
visual 0.0011637431350344871
linguistic 0.00011861228107082273
